# Receipt Detail ML PdOI_ATG

## Import Libraries

In [1]:
from pandas import set_option,concat,DataFrame,Series,to_numeric,options
from numpy import concatenate,round
from pyodbc import connect
from warnings import filterwarnings
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error,mean_squared_error,median_absolute_error,r2_score,max_error
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,HistGradientBoostingRegressor 
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor#,plot_tree
from sklearn.model_selection import GridSearchCV,train_test_split#,cross_val_score
from sklearn.preprocessing import StandardScaler#,LabelBinarizer,label_binarize,MaxAbsScaler
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.neighbors import *
from re import search#,match
from random import randint
from time import time
from smtplib import SMTP_SSL
from ssl import create_default_context
from os import path,walk
from zipfile import ZipFile, ZIP_DEFLATED
from time import time,sleep

In [2]:
filterwarnings('ignore')
set_option('display.max_rows', None)
set_option('display.max_columns', None)

## Connect to SQL

In [3]:
server = 'MOE'
database = 'prod_WeisMarkets_RecoverNow'
connectionString = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Integrated Security={True};Autocommit={True};Trusted_Connection=yes;'
conn = connect(connectionString)
cursor_1 = conn.cursor()

## Supporting Functions

In [4]:
def export_to_sql(data: DataFrame):
    server = 'barney'
    database = 'sandbox_mp'
    connectionString = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Integrated Security={True};Autocommit={True};Trusted_Connection=yes;'
    conn = connect(connectionString)
    cursor = conn.cursor()
    for index in range(len(list(data.columns))):
        if(not(index in [202,201,200,199,198,197,196,195,181])):
            data[list(data.columns)[index]] = data[list(data.columns)[index]].astype(str)
        else:
            data[list(data.columns)[index]] = data[list(data.columns)[index]].astype(float)
            data[list(data.columns)[index]] = data[list(data.columns)[index]].round(4)
    insert_sql = """
        INSERT INTO Weis_Market_Claim_With_ML_Predictions_PdOI_ATG (
			ATG_RCPT_DTL_DS_Checked,
			ATG_RCPT_DTL_DE_Checked,
			ATG_RCPT_DTL_ClientFamilyID_ATG,
			ATG_RCPT_DTL_BusinessUnit,
			ATG_RCPT_DTL_ReceiptNbr,
			ATG_RCPT_DTL_ReceiptSfx,
			ATG_RCPT_DTL_PONbr,
			ATG_RCPT_DTL_PODate,
			ATG_RCPT_DTL_ReceiptDate,
			ATG_RCPT_DTL_POVendorNbr,
			ATG_RCPT_DTL_GrossWght,
			ATG_RCPT_DTL_UnitOfWeight,
			ATG_RCPT_DTL_POQty,
			ATG_RCPT_DTL_ReceiptQty,
			ATG_RCPT_DTL_ShortQty_ATG,
			ATG_RCPT_DTL_VarWghtInd,
			ATG_RCPT_DTL_ItemNbr,
			ATG_RCPT_DTL_ItemDescr,
			ATG_RCPT_DTL_Dept,
			ATG_RCPT_DTL_UPCNbr,
			ATG_RCPT_DTL_ItemSize,
			ATG_RCPT_DTL_CasePack,
			ATG_RCPT_DTL_TurnQty_ATG,
			ATG_RCPT_DTL_TurnUnitQty_ATG,
			ATG_RCPT_DTL_TurnRatio_ATG,
			ATG_RCPT_DTL_ListCostSB_ATG,
			ATG_RCPT_DTL_ChangeDate_ATG,
			ATG_RCPT_DTL_EndDate_ATG,
			ATG_RCPT_DTL_NextCost_ATG,
			ATG_RCPT_DTL_PrevCost_ATG,
			ATG_RCPT_DTL_CostDiff_ATG,
			ATG_RCPT_DTL_IncDecFlag_ATG,
			ATG_RCPT_DTL_CostType_ATG,
			ATG_RCPT_DTL_AddDate_ATG,
			ATG_RCPT_DTL_SellEffDate_ATG,
			ATG_RCPT_DTL_LastShipDate_ATG,
			ATG_RCPT_DTL_ATG_Cost_Ref,
			ATG_RCPT_DTL_PdGross_ATG,
			ATG_RCPT_DTL_PdNet_ATG,
			ATG_RCPT_DTL_PdQty_ATG,
			ATG_RCPT_DTL_PdUnitGross_ATG,
			ATG_RCPT_DTL_PdUnitNet_ATG,
			ATG_RCPT_DTL_PdUnitQty_ATG,
			ATG_RCPT_DTL_Facility,
			ATG_RCPT_DTL_ATG_Hdr_Ref,
			ATG_RCPT_DTL_SourceFile_ATG,
			ATG_RCPT_DTL_ATG_Ref,
			ATG_RCPT_DTL_ATG_ItemID,
			ATG_RCPT_DTL_RcptFacility,
			ATG_RCPT_DTL_APVendorNbr,
			ATG_RCPT_DTL_Merchndsr,
			ATG_RCPT_DTL_ShipWght,
			ATG_RCPT_DTL_ShipCube,
			ATG_RCPT_DTL_PalltQty,
			ATG_RCPT_DTL_QualifyAmt,
			ATG_RCPT_DTL_RepckRatio,
			ATG_RCPT_DTL_Status,
			ATG_RCPT_DTL_ExcptnDate,
			ATG_RCPT_DTL_OKDate,
			ATG_RCPT_DTL_AdjCde1,
			ATG_RCPT_DTL_AdjQty1,
			ATG_RCPT_DTL_AdjCde2,
			ATG_RCPT_DTL_AdjQty2,
			ATG_RCPT_DTL_ListCost,
			ATG_RCPT_DTL_OI,
			ATG_RCPT_DTL_FreeGds,
			ATG_RCPT_DTL_WhseDisc,
			ATG_RCPT_DTL_UpDnAmt,
			ATG_RCPT_DTL_UpDnInd,
			ATG_RCPT_DTL_BB,
			ATG_RCPT_DTL_LastCost,
			ATG_RCPT_DTL_LastCost_Orig,
			ATG_RCPT_DTL_CashDisc,
			ATG_RCPT_DTL_FrghtAllw,
			ATG_RCPT_DTL_FrghtAllwInd,
			ATG_RCPT_DTL_VndrUpDn,
			ATG_RCPT_DTL_VndrUpDnInd,
			ATG_RCPT_DTL_PPayAdd,
			ATG_RCPT_DTL_PrePayAddExInd,
			ATG_RCPT_DTL_FrghtBill,
			ATG_RCPT_DTL_Bckhl,
			ATG_RCPT_DTL_VarWght,
			ATG_RCPT_DTL_InvQty,
			ATG_RCPT_DTL_InvListCost,
			ATG_RCPT_DTL_InvOI,
			ATG_RCPT_DTL_InvFreeGds,
			ATG_RCPT_DTL_InvWhseDisc,
			ATG_RCPT_DTL_InvUpDnAmt,
			ATG_RCPT_DTL_InvUpDnInd,
			ATG_RCPT_DTL_InvBB,
			ATG_RCPT_DTL_InvLastCost,
			ATG_RCPT_DTL_InvLastCost_Orig,
			ATG_RCPT_DTL_InvCashDisc,
			ATG_RCPT_DTL_InvFrghtAllw,
			ATG_RCPT_DTL_InvFrghtAllwExInd,
			ATG_RCPT_DTL_InvVndrUpDn,
			ATG_RCPT_DTL_InvVndrUpDnInd,
			ATG_RCPT_DTL_InvPPayAdd,
			ATG_RCPT_DTL_InvPPayAddExInd,
			ATG_RCPT_DTL_InvFrghtBill,
			ATG_RCPT_DTL_InvBckhl,
			ATG_RCPT_DTL_InvWght,
			ATG_RCPT_DTL_InvVarWght,
			ATG_RCPT_DTL_AdjQty,
			ATG_RCPT_DTL_AdjListCost,
			ATG_RCPT_DTL_AdjOI,
			ATG_RCPT_DTL_AdjFreeGds,
			ATG_RCPT_DTL_AdjWhseDisc,
			ATG_RCPT_DTL_AdjUpDnAmt,
			ATG_RCPT_DTL_AdjBB,
			ATG_RCPT_DTL_AdjLastCost,
			ATG_RCPT_DTL_AdjLastCost_Orig,
			ATG_RCPT_DTL_AdjFrghtAllw,
			ATG_RCPT_DTL_AdjVndrUpDn,
			ATG_RCPT_DTL_AdjPPayAdd,
			ATG_RCPT_DTL_AdjFrghtBill,
			ATG_RCPT_DTL_AdjBckhl,
			ATG_RCPT_DTL_AdjWght,
			ATG_RCPT_DTL_AdjVarWght,
			ATG_RCPT_DTL_AdjUpDnInd,
			ATG_RCPT_DTL_AdjVndrUpDnInd,
			ATG_RCPT_DTL_Toggle,
			ATG_RCPT_DTL_LastRcvCorrDate,
			ATG_RCPT_DTL_Comment,
			ATG_RCPT_DTL_FreeCs,
			ATG_RCPT_DTL_Trans,
			ATG_RCPT_DTL_TransQty,
			ATG_RCPT_DTL_Trans2RcvNbr,
			ATG_RCPT_DTL_Trans2RcvSfx,
			ATG_RCPT_DTL_Trans2Qty,
			ATG_RCPT_DTL_TransFromRcvNbr,
			ATG_RCPT_DTL_TransFromRcvSfx,
			ATG_RCPT_DTL_TransFromQty,
			ATG_RCPT_DTL_ItmFrtAllwExInd,
			ATG_RCPT_DTL_AdjPPayAddExInd,
			ATG_RCPT_DTL_FrghtBillExInd,
			ATG_RCPT_DTL_InvFrghtBillExInd,
			ATG_RCPT_DTL_AdjFrghtBillExInd,
			ATG_RCPT_DTL_BckhlExInd,
			ATG_RCPT_DTL_InvBckhlExInd,
			ATG_RCPT_DTL_AdjBckhlExInd,
			ATG_RCPT_DTL_DealFlg,
			ATG_RCPT_DTL_DealStatus,
			ATG_RCPT_DTL_InvQualifyAmt,
			ATG_RCPT_DTL_InvFreeCs,
			ATG_RCPT_DTL_AdjQualifyAmt,
			ATG_RCPT_DTL_AdjFreeCs,
			ATG_RCPT_DTL_CsUnitFctr,
			ATG_RCPT_DTL_InvSurchgInd,
			ATG_RCPT_DTL_HiOldAvgCost,
			ATG_RCPT_DTL_ACChgTkn,
			ATG_RCPT_DTL_ExcptnUsrID,
			ATG_RCPT_DTL_OKUsrID,
			ATG_RCPT_DTL_WDCostFlg,
			ATG_RCPT_DTL_OICostFlg,
			ATG_RCPT_DTL_FGCostFlg,
			ATG_RCPT_DTL_FACostFlg,
			ATG_RCPT_DTL_PPAddCostFlg,
			ATG_RCPT_DTL_BBCostFlg,
			ATG_RCPT_DTL_UDCostFlg,
			ATG_RCPT_DTL_VndUDCostFlg,
			ATG_RCPT_DTL_FBCostFlg,
			ATG_RCPT_DTL_BHCostFlg,
			ATG_RCPT_DTL_FullCostFlg,
			ATG_RCPT_DTL_DSDItmDueVndr,
			ATG_RCPT_DTL_ATG_Ref_Orig,
			ATG_RCPT_DTL_ItemFacility,
			ATG_RCPT_DTL_ATG_PO_Ref,
			ATG_RCPT_DTL_IncvOI_ATG,
			ATG_RCPT_DTL_IncvBB_ATG,
			ATG_RCPT_DTL_FlatAmtOI_ATG,
			ATG_RCPT_DTL_FlatAmtBB_ATG,
			ATG_RCPT_DTL_Contact,
			ATG_RCPT_DTL_BestOI_ATG,
			ATG_RCPT_DTL_BestBB_ATG,
			ATG_RCPT_DTL_UPCUnit,
			ATG_RCPT_DTL_BalFlag_ATG,
			ATG_RCPT_DTL_OOB_ATG,
			ATG_RCPT_DTL_PdHdrFrtVar_ATG,
			ATG_RCPT_DTL_MiscAdj_ATG,
			ATG_RCPT_DTL_PdOI_ATG_BeforeAI,
			ATG_RCPT_DTL_PdBB_ATG,
			ATG_RCPT_DTL_PdUpDn_ATG,
			ATG_RCPT_DTL_PdIncvOI_ATG,
			ATG_RCPT_DTL_PdFlatOI_ATG,
			ATG_RCPT_DTL_PdFlatBB_ATG,
			ATG_RCPT_DTL_PdBckHl_ATG,
			ATG_RCPT_DTL_PdPPayAdd_ATG,
			ATG_RCPT_DTL_PdFrtBil_ATG,
			ATG_RCPT_DTL_PdFrtAlw_ATG,
			ATG_RCPT_DTL_PdShortOI_ATG,
			ATG_RCPT_DTL_PdShortBB_ATG,
			ATG_RCPT_DTL_PdShortBckHl_ATG,
			Linear_Model_Predictions_ATG_RCPT_DTL_PdOI_ATG,
			DecisionTree_Model_Predictions_ATG_RCPT_DTL_PdOI_ATG,
			RandomForest_Model_Predictions_ATG_RCPT_DTL_PdOI_ATG,
			XGBoost_Model_Predictions_ATG_RCPT_DTL_PdOI_ATG,
			GradientBoost_Model_Predictions_ATG_RCPT_DTL_PdOI_ATG,
			LGBM_Model_Predictions_ATG_RCPT_DTL_PdOI_ATG,
			CatBoost_Model_Predictions_ATG_RCPT_DTL_PdOI_ATG,
			KNeighbors_Model_Predictions_ATG_RCPT_DTL_PdOI_ATG
        )
        VALUES 
            (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
            ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
            ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
            ?,?,?,?,?,?,?,?) 
    """
    for row in data.index:
        if(int(row)%100==0):
            print(f"{row:,}")
        current_row = list(data.iloc[int(row)])
        # for item in current_row:
        #     print(f"{item}: {type(item)}")
        # del current_row[33]
        # for index in range(len(current_row)):
        #     if(index>=33):
        #         current_row[index] = round(current_row[index],4)
        #     if(index in [2,3,4]):
        #         current_row[index] = int(current_row[index])
        #     else:
        #         current_row[index] = str(current_row[index])
        # current_row.insert(33,current_row[23])
        # del current_row[23]
        try:
            cursor.execute(insert_sql, current_row)
            conn.commit()
        except:
            pass
    cursor.close()
    conn.close()

In [5]:
def scatter_plot_models(target: Series,predictions: Series,model_name: str, show: bool=False):
    plt.figure(figsize=(6,5))
    plt.scatter(target,predictions,color='blue',label='Test_Data',linewidths=.1)
    plt.plot([Series(target).min(),Series(target).max()],
            [Series(target).min(),Series(target).max()],'k--',lw=1)
    plt.xlabel('PdOI_ATG_Before_AI')
    plt.ylabel('PdOI_ATG_After_AI')
    plt.savefig(f"C:/Code/Python/Machine_Learning_AI/Model_Analysis/{model_name}_Scatter.png")
    if(show):
        plt.show()
    else:
        plt.close()

In [6]:
def get_scores(target, predictions):
    mse = mean_squared_error(target, predictions)
    rmse = mse**0.5
    median_ae = median_absolute_error(target,predictions)
    mean_ae = mean_absolute_error(target,predictions)
    max_errors = max_error(target,predictions)
    r2 = r2_score(target,predictions)
    return [mse,rmse,mean_ae,median_ae,max_errors,r2]

In [7]:
def run_and_analyze_model(model,features_train_subset: DataFrame,target_train_subset: Series,features_test: Series,target_test: Series,model_scores: DataFrame,model_name: str):
    model.fit(features_train_subset,target_train_subset)
    train_predictions = Series(model.predict(features_train_subset),name=model_name)
    train_predictions = train_predictions.round(4)
    predictions = Series(model.predict(features_test),name=model_name)
    predictions = predictions.round(4)
    model_scores.loc[model_name] = get_scores(target_test,predictions)
    try:
        model_importances = DataFrame([model.feature_importances_],columns=features_train_subset.columns,index=[model_name]).T
        return model,train_predictions,predictions,model_scores,model_importances
    except:
        model_importances = DataFrame([[1/len(features_train_subset.columns) for _ in range(len(features_train_subset.columns))]],columns=list(features_train_subset.columns),index=[model_name]).T
        return model,train_predictions,predictions,model_scores,model_importances

In [8]:
def empty_string_to_null(string: str):
    if(len(str(string))==0 or len(str(string).replace(' ',''))==0):
        return "None"
    return string

In [9]:
def object_to_int(original,lookup_table):
    for row in lookup_table.index:
        for col in lookup_table.columns:
            if(original==lookup_table[col][row]):
                return int(col)

In [10]:
def random_int(min_val, max_val):
    # Get the current time in microseconds
    current_time = int(time() * 1000000)
    
    # Use the current time as a seed and perform some operations to get more randomness
    seed = (current_time ^ (current_time >> randint(1,20))) & 0xFFFFFFFF
    seed = (seed ^ (seed << randint(1,20))) & 0xFFFFFFFF
    seed = (seed ^ (seed >> randint(1,20))) & 0xFFFFFFFF
    
    # Scale the seed to the desired range
    random_value = min_val + (seed % (max_val - min_val + 1))
    
    return random_value

In [11]:
def random_features(columns):
    subset = []
    columns = list(columns)
    while(len(subset)<50):
        random_index = random_int(0,len(list(columns))-1)
        while(columns[random_index] in subset):
            random_index = random_int(0,len(list(columns))-1)
        subset.append(columns[random_index])
    return subset

In [12]:
def send_email_to_self(subject: str,body: str):
    port = 465  # For SSL
    smtp_server = "smtp.gmail.com"
    sender_email = "micpowers98@gmail.com"  # Enter your address
    receiver_email = "micpowers98@gmail.com"  # Enter receiver address
    password = 'efex cwhv gppq ueob'
    message = "Subject: "+subject+"\n"+body

    context = create_default_context()
    with SMTP_SSL(smtp_server, port, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, message)

In [13]:
def get_data(columns: list):
	cursor_1.execute("""
			select 
				case when (li.checked='true' and ds.checked='claim') then 'Y' else 'N' end as IsClaimFlg, li.checked,
					li.[ATG_Ref],
					li.[ATG_LI_Ref],
					li.[BatchNbr],
					li.[CATEGORY_ATG],
					li.[ClaimType],
					li.[AP_VndNbr],
					li.[APVndrName],
					li.[OOB_ATG],
					li.[ItemNbr],
					li.[UPCNbr],
					li.[UPCUnit],
					li.[ItemDescription],
					li.[ItemShipPack],
					li.[PoNbr],
					li.[PODate],
					li.[ReceivingDate],
					li.[TurnRatio_ATG],
					li.[TurnQty_ATG],
					li.[OrdQty],
					li.[PdQty_ATG],
					li.[PdGross_ATG],
					li.[PdOI_ATG],
					li.[PdBB_ATG],
					li.[PdNet_ATG],
					li.[DealNbr],
					li.[OrdStartDate_ATG],
					li.[OrdEndDate_ATG],
					li.[DateStartArrival_ATG],
					li.[DateEndArrival_ATG],
					li.[DLAmtOI],
					li.[DLAmtBB]
			from     
				[MOE].[prod_WeisMarkets_RecoverNow].[dbo].[ATG_DealLI_Exceptions] li
						join  [MOE].[prod_WeisMarkets_RecoverNow].[dbo].[ATG_Deal_Summary] ds
						on (li.BatchNbr = ds.BatchNbr)
						and (li.DLVendorNbr = ds.DLVendorNbr)
						and (li.DealNbr = ds.DealNbr)
						and (li.CATEGORY_ATG = ds.CATEGORY_ATG)
						and (li.ClaimType = ds.ClaimType_ATG)
			where li.ClaimType = 'IN DEAL' and ds.CATEGORY_ATG = 'SAME VENDOR - AMT DEALS'
			order by 8,12,17			
		""")
	train_subset_data = cursor_1.fetchall()
	train_subset_data_list = []
	for index in range(len(train_subset_data)):
		train_subset_data_list.append(list(train_subset_data[index]))
	del train_subset_data
	return DataFrame(data=train_subset_data_list,columns=columns)

In [14]:
def get_big_data(columns: list):
	cursor_1.execute("""
            SELECT DS.checked as DS_Checked, DE.checked as DE_Checked, RD.*
                FROM 
                    MOE.prod_WeisMarkets_RecoverNow.dbo.ATG_RCPT_DTL as RD LEFT OUTER JOIN
                        MOE.prod_WeisMarkets_RecoverNow.dbo.ATG_DealLI_Exceptions_TBL as DE 
                        ON RD.ATG_Ref = DE.ATG_LI_Ref LEFT OUTER JOIN
                        MOE.prod_WeisMarkets_RecoverNow.dbo.ATG_Deal_Summary_TBL as DS
                            ON DE.ClaimType = DS.ClaimType_ATG
                                AND DE.DealNbr = DS.DealNbr
                                AND DE.DLVendorNbr = DS.DLVendorNbr
                                AND DE.BatchNbr = DS.BatchNbr		
		""")
	train_subset_data = cursor_1.fetchall()
	train_subset_data_list = []
	for index in range(len(train_subset_data)):
		train_subset_data_list.append(list(train_subset_data[index]))
	del train_subset_data
	return DataFrame(data=train_subset_data_list,columns=columns)

In [15]:
def get_individual_upc(columns: list, upc: str):
    cursor_1.execute(f"""
            SELECT DS.checked as DS_Checked, DE.checked as DE_Checked, RD.*
                FROM 
                    MOE.prod_WeisMarkets_RecoverNow.dbo.ATG_RCPT_DTL as RD LEFT OUTER JOIN
                        MOE.prod_WeisMarkets_RecoverNow.dbo.ATG_DealLI_Exceptions_TBL as DE 
                        ON RD.ATG_Ref = DE.ATG_LI_Ref LEFT OUTER JOIN
                        MOE.prod_WeisMarkets_RecoverNow.dbo.ATG_Deal_Summary_TBL as DS
                            ON DE.ClaimType = DS.ClaimType_ATG
                                AND DE.DealNbr = DS.DealNbr
                                AND DE.DLVendorNbr = DS.DLVendorNbr
                                AND DE.BatchNbr = DS.BatchNbr
                WHERE
                    RD.UPCNbr={upc}
		""")
    train_subset_data = cursor_1.fetchall()
    train_subset_data_list = []
    for index in range(len(train_subset_data)):
        train_subset_data_list.append(list(train_subset_data[index]))
    del train_subset_data
    #print(DataFrame(data=train_subset_data_list,columns=columns).info(max_cols=1000))
    train = DataFrame(data=train_subset_data_list,columns=columns)
    train = train.drop('ClaimActivityID_ATGSYS',axis=1)
    train = train.fillna('None')
    return train

In [16]:
def get_test_data(batch_number: int, columns: list):
    cursor_1.execute(f"""
		    select 
                case when (li.checked = 'true' and ds.checked = 'claim') then 'Y' else 'N' end as IsClaimFlg, li.Checked, 
                    li.[ATG_Ref],
                    li.[ATG_LI_Ref],
                    li.[BatchNbr],
                    li.[CATEGORY_ATG],
                    li.[ClaimType],
                    li.[AP_VndNbr],
                    li.[APVndrName],
                    li.[OOB_ATG],
                    li.[ItemNbr],
                    li.[UPCNbr],
                    li.[UPCUnit],
                    li.[ItemDescription],
                    li.[ItemShipPack],
                    li.[PoNbr],
                    li.[PODate],
                    li.[ReceivingDate],
                    li.[TurnRatio_ATG],
                    li.[TurnQty_ATG],
                    li.[OrdQty],
                    li.[PdQty_ATG],
                    li.[PdGross_ATG],
                    li.[PdOI_ATG],
                    li.[PdBB_ATG],
                    li.[PdNet_ATG],
                    li.[DealNbr],
                    li.[OrdStartDate_ATG],
                    li.[OrdEndDate_ATG],
                    li.[DateStartArrival_ATG],
                    li.[DateEndArrival_ATG],
                    li.[DLAmtOI],
                    li.[DLAmtBB]--,
			    from    
                    [MOE].[prod_WeisMarkets_RecoverNow].[dbo].[ATG_DealLI_Exceptions] li
                    join  [MOE].[prod_WeisMarkets_RecoverNow].[dbo].[ATG_Deal_Summary]  ds
                    on (li.BatchNbr = ds.BatchNbr)
                    and (li.DLVendorNbr = ds.DLVendorNbr)
                    and (li.DealNbr = ds.DealNbr)
                    and (li.CATEGORY_ATG = ds.CATEGORY_ATG)
                    and (li.ClaimType = ds.ClaimType_ATG)
                where
                    li.BatchNbr=67
                    and
                    li.ClaimType = 'IN DEAL' 
                    and 
                    ds.CATEGORY_ATG = 'SAME VENDOR - AMT DEALS' 
                    and 
                    li.checked='x' 
                    and 
                    ds.checked='x'

		    union

			select top(1) 
                case when (li.checked = 'true' and ds.checked = 'claim') then 'Y' else 'N' end as IsClaimFlg, li.Checked, 
                    li.[ATG_Ref],
                    li.[ATG_LI_Ref],
                    li.[BatchNbr],
                    li.[CATEGORY_ATG],
                    li.[ClaimType],
                    li.[AP_VndNbr],
                    li.[APVndrName],
                    li.[OOB_ATG],
                    li.[ItemNbr],
                    li.[UPCNbr],
                    li.[UPCUnit],
                    li.[ItemDescription],
                    li.[ItemShipPack],
                    li.[PoNbr],
                    li.[PODate],
                    li.[ReceivingDate],
                    li.[TurnRatio_ATG],
                    li.[TurnQty_ATG],
                    li.[OrdQty],
                    li.[PdQty_ATG],
                    li.[PdGross_ATG],
                    li.[PdOI_ATG],
                    li.[PdBB_ATG],
                    li.[PdNet_ATG],
                    li.[DealNbr],
                    li.[OrdStartDate_ATG],
                    li.[OrdEndDate_ATG],
                    li.[DateStartArrival_ATG],
                    li.[DateEndArrival_ATG],
                    li.[DLAmtOI],
                    li.[DLAmtBB]--,
			    from     
                    [MOE].[prod_WeisMarkets_RecoverNow].[dbo].[ATG_DealLI_Exceptions] li
                    join  [MOE].[prod_WeisMarkets_RecoverNow].[dbo].[ATG_Deal_Summary]  ds
                    on (li.BatchNbr = ds.BatchNbr)
                    and (li.DLVendorNbr = ds.DLVendorNbr)
                    and (li.DealNbr = ds.DealNbr)
                    and (li.CATEGORY_ATG = ds.CATEGORY_ATG)
                    and (li.ClaimType = ds.ClaimType_ATG)
			where
                li.ClaimType = 'IN DEAL' 
                and 
                ds.CATEGORY_ATG = 'SAME VENDOR - AMT DEALS' 
                and 
                li.checked='true' 
                and 
                ds.checked='claim'
        """)
    test_data = cursor_1.fetchall()
    test_data_list = []
    for index in range(len(test_data)):
        test_data_list.append(list(test_data[index]))
    del test_data
    return DataFrame(data=test_data_list,columns=columns)

In [17]:
def compress_folder_to_zip(folder_path, output_zip_file):
    # Create a ZipFile object
    with ZipFile(output_zip_file, 'w', ZIP_DEFLATED) as zipf:
        # Walk through the folder
        for root, dirs, files in walk(folder_path):
            for file in files:
                # Create the full file path
                file_path = path.join(root, file)
                # Add file to zip, using relative path to maintain folder structure
                arcname = path.relpath(file_path, start=folder_path)
                zipf.write(file_path, arcname=arcname)

## Global Variables

In [18]:
old_columns:list = ['IsClaimFlg','Checked','ATG_Ref','ATG_LI_Ref','BatchNbr','CATEGORY_ATG','ClaimType','AP_VndNbr','APVndrName',
                 'OOB_ATG','ItemNbr','UPCNbr','UPCUnit','ItemDescription','ItemShipPack','PoNbr','PODate','ReceivingDate','TurnRatio_ATG',
                 'TurnQty_ATG','OrdQty','PdQty_ATG','PdGross_ATG','PdOI_ATG','PdBB_ATG','PdNet_ATG','DealNbr','OrdStartDate_ATG',
                 'OrdEndDate_ATG','DateStartArrival_ATG','DateEndArrival_ATG','DLAmtOI','DLAmtBB']

In [19]:
columns:list = [
        'DS_Checked','DE_Checked','ClientFamilyID_ATG','BusinessUnit','ReceiptNbr','ReceiptSfx','PONbr','PODate','ReceiptDate','POVendorNbr',
        'GrossWght','UnitOfWeight','POQty','ReceiptQty','ShortQty_ATG','VarWghtInd','ItemNbr','ItemDescr','Dept','UPCNbr',
        'ItemSize','CasePack','TurnQty_ATG','TurnUnitQty_ATG','TurnRatio_ATG','ListCostSB_ATG','ChangeDate_ATG','EndDate_ATG','NextCost_ATG','PrevCost_ATG',
        'CostDiff_ATG','IncDecFlag_ATG','CostType_ATG','AddDate_ATG','SellEffDate_ATG','LastShipDate_ATG','ATG_Cost_Ref','PdGross_ATG','PdNet_ATG','PdQty_ATG',
        'PdUnitGross_ATG','PdUnitNet_ATG','PdUnitQty_ATG','Facility','ATG_Hdr_Ref','SourceFile_ATG','ATG_Ref','ATG_ItemID','RcptFacility','APVendorNbr',
        'Merchndsr','ShipWght','ShipCube','PalltQty','QualifyAmt','RepckRatio','Status','ExcptnDate','OKDate','AdjCde1',
        'AdjQty1','AdjCde2','AdjQty2','ListCost','OI','FreeGds','WhseDisc','UpDnAmt','UpDnInd','BB',
        'LastCost','LastCost_Orig','CashDisc','FrghtAllw','FrghtAllwInd','VndrUpDn','VndrUpDnInd','PPayAdd','PrePayAddExInd','FrghtBill',
        'Bckhl','VarWght','InvQty','InvListCost','InvOI','InvFreeGds','InvWhseDisc','InvUpDnAmt','InvUpDnInd','InvBB',
        'InvLastCost','InvLastCost_Orig','InvCashDisc','InvFrghtAllw','InvFrghtAllwExInd','InvVndrUpDn','InvVndrUpDnInd','InvPPayAdd','InvPPayAddExInd','InvFrghtBill',
        'InvBckhl','InvWght','InvVarWght','AdjQty','AdjListCost','AdjOI','AdjFreeGds','AdjWhseDisc','AdjUpDnAmt','AdjBB',
        'AdjLastCost','AdjLastCost_Orig','AdjFrghtAllw','AdjVndrUpDn','AdjPPayAdd','AdjFrghtBill','AdjBckhl','AdjWght','AdjVarWght','AdjUpDnInd',
        'AdjVndrUpDnInd','Toggle','LastRcvCorrDate','Comment','FreeCs','Trans','TransQty','Trans2RcvNbr','Trans2RcvSfx','Trans2Qty',
        'TransFromRcvNbr','TransFromRcvSfx','TransFromQty','ItmFrtAllwExInd','AdjPPayAddExInd','FrghtBillExInd','InvFrghtBillExInd','AdjFrghtBillExInd','BckhlExInd','InvBckhlExInd',
        'AdjBckhlExInd','DealFlg','DealStatus','InvQualifyAmt','InvFreeCs','AdjQualifyAmt','AdjFreeCs','CsUnitFctr','InvSurchgInd','HiOldAvgCost',
        'ACChgTkn','ExcptnUsrID','OKUsrID','WDCostFlg','OICostFlg','FGCostFlg','FACostFlg','PPAddCostFlg','BBCostFlg','UDCostFlg',
        'VndUDCostFlg','FBCostFlg','BHCostFlg','FullCostFlg','DSDItmDueVndr','ATG_Ref_Orig','ItemFacility','ATG_PO_Ref','IncvOI_ATG','IncvBB_ATG',
        'FlatAmtOI_ATG','FlatAmtBB_ATG','Contact','BestOI_ATG','BestBB_ATG','UPCUnit','BalFlag_ATG','OOB_ATG','PdHdrFrtVar_ATG','MiscAdj_ATG',
        'PdOI_ATG','PdBB_ATG','PdUpDn_ATG','PdIncvOI_ATG','PdFlatOI_ATG','PdFlatBB_ATG','PdBckHl_ATG','PdPPayAdd_ATG','PdFrtBil_ATG','PdFrtAlw_ATG',
        'PdShortOI_ATG','PdShortBB_ATG','PdShortBckHl_ATG','ClaimActivityID_ATGSYS'
    ]

In [20]:
additional_excluded_columns:list = ['BusinessUnit','ReceiptSfx','UnitOfWeight','VarWghtInd','ItemSize','SourceFile_ATG','ATG_Ref','ATG_ItemID',
                                   'RcptFacility','Merchndsr','Status','ExcptnDate','OKDate','AdjCde1','AdjQty1','AdjCde2','AdjQty2','ListCost','FreeGds',
                                   'WhseDisc','UpDnAmt','UpDnInd','BB','LastCost','LastCost_Orig','CashDisc','FrghtAllwInd','VndrUpDn','VndrUpDnInd',
                                   'PPayAdd','PrePayAddExInd','FrghtBill','Bckhl','VarWght','InvQty','InvListCost','InvOI','InvFreeGds','InvWhseDisc',
                                   'InvUpDnInd','InvBB','InvLastCost','InvLastCost_Orig','InvCashDisc','InvFrghtAllw','InvFrghtAllwExInd','InvVndrUpDn',
                                   'InvPPayAdd','InvFrghtBill','InvBckhl','InvWght','InvVarWght','AdjQty','AdjListCost','AdjOI','AdjFreeGds','AdjWhseDisc',
                                   'AdjUpDnAmt','AdjBB','AdjLastCost','AdjLastCost_Orig','AdjFrghtAllw','DS_Checked','DE_Checked','ClientFamilyID_ATG',
                                   'AdjVndrUpDn','AdjPPayAdd','AdjFrghtBill','AdjBckhl','AdjWght','AdjVarWght','AdjUpDnInd','AdjVndrUpDnInd','Toggle',
                                   'LastRcvCorrDate','Comment','FreeCs','Trans','TransQty','Trans2RcvNbr','Trans2RcvSfx','Trans2Qty','TransFromRcvNbr',
                                   'TransFromRcvSfx','TransFromQty','ItmFrtAllwExInd','AdjPPayAddExInd','FrghtBillExInd','InvFrghtBillExInd','AdjFrghtBillExInd',
                                   'BckhlExInd','InvBckhlExInd','AdjBckhlExInd','DealFlg','DealStatus','InvQualifyAmt','InvFreeCs',
                                   'AdjQualifyAmt','AdjFreeCs','CsUnitFctr','InvSurchgInd','HiOldAvgCost','ACChgTkn','ExcptnUsrID','OKUsrID','WDCostFlg',
                                   'OICostFlg','FGCostFlg','FACostFlg','PPAddCostFlg','BBCostFlg','UDCostFlg','VndUDCostFlg','FBCostFlg','BHCostFlg',
                                   'FullCostFlg','DSDItmDueVndr','ATG_Ref_Orig','ItemFacility','ATG_PO_Ref','IncvOI_ATG','IncvBB_ATG','FlatAmtOI_ATG',
                                   'FlatAmtBB_ATG','Contact','BalFlag_ATG','OOB_ATG','PdHdrFrtVar_ATG','MiscAdj_ATG','ReceiptNbr','PONbr','ReceiptDate',
                                   'POVendorNbr','ItemDescr','ItemNbr','Dept','UPCNbr','ChangeDate_ATG','EndDate_ATG','NextCost_ATG','PrevCost_ATG',
                                   'CostDiff_ATG','IncDecFlag_ATG','CostType_ATG','AddDate_ATG','SellEffDate_ATG','LastShipDate_ATG','ATG_Cost_Ref',
                                   'Facility','ATG_Hdr_Ref','APVendorNbr','InvUpDnAmt','InvVndrUpDnInd','InvPPayAddExInd','UPCUnit','ListCostSB_ATG',
                                   'RepckRatio','QualifyAmt','ShipCube','BestOI_ATG','OI','PODate','PdIncvOI_ATG','PdUpDn_ATG','PdShortOI_ATG','PdFlatOI_ATG']

## Loading Data

In [21]:
train: DataFrame = get_big_data(columns=columns)

In [ ]:
print(f"{len(train):,}")

In [ ]:
train_subset_original: DataFrame = train.copy()

In [ ]:
train_subset: DataFrame = train.copy()
train = train.drop(additional_excluded_columns,axis=1)

In [ ]:
print(f"{len(train_subset):,} total rows of data.")

In [ ]:
subject = 'Data Collected'
body = f"""
        {len(train_subset):,} total rows of data.
    """
# send_email_to_self(subject,body)

## Convert Strings to Numbers

In [ ]:
for col in train_subset.columns:
    if train_subset[col].dtype == 'object':
        try:
            train_subset[col] = train_subset[col].astype(float)
        except:
            try:
                train_subset[col] = train_subset[col].astype(int)
            except Exception as e: pass
print(f"Initial conversions from strings to numbers")

In [ ]:
empty_col = []
for col in train_subset.columns:
    if train_subset[col].dtype == 'object':
        if(train_subset[col].nunique()>1000):
            print(col)
            empty_col.append(col)

In [ ]:
train_subset = train_subset.drop(empty_col,axis=1)

In [ ]:
for col in train_subset.columns:
    if train_subset[col].dtype == 'object' and not(col=='PdOI_ATG'):
        try:
            train_subset[col] = train_subset[col].astype(float)
        except:
            try:
                train_subset[col] = train_subset[col].astype(int)
            except Exception as e: pass

## Fill NULL Values

In [ ]:
for col in train_subset.columns:
    if train_subset[col].dtype == 'object':
        train_subset[col] = train_subset[col].fillna('None')
    elif train_subset[col].dtype == 'int64':
        train_subset[col] = train_subset[col].fillna(-1)
    elif train_subset[col].dtype == 'float64':
        train_subset[col] = train_subset[col].fillna(-1)

In [ ]:
for col in train_subset.columns:
    if train_subset[col].dtype == 'object' and not(col=='PdOI_ATG'):
        try:
            train_subset[col] = train_subset[col].astype(float)
        except:
            try:
                train_subset[col] = train_subset[col].astype(int)
            except Exception as e: pass

## Creating Categories

In [ ]:
category_values: dict = {}
category_new_values: dict = {}
for col in train_subset.columns:
    if train_subset[col].dtype == 'object' and train_subset[col].nunique()<=1000 and not(col=='PdOI_ATG'):
        train_subset[col] = train_subset[col].apply(empty_string_to_null)
        # categorical_columns.append(col)
        category_values[col] = Series(list(train_subset[col].unique())).unique()

In [ ]:
max_length = 0
for key,item in category_values.items():
    if(len(item)>max_length):
        max_length = len(item)

In [ ]:
for key in category_values:
    category_values[key] = concatenate((category_values[key],['None'] * (max_length - len(category_values[key]))))

In [ ]:
lookup_table = DataFrame.from_dict(category_values, orient='index')
lookup_table.columns = range(max_length)

In [ ]:
for col in lookup_table.columns:
    lookup_table[col] = lookup_table[col].astype(str)

In [ ]:
lookup_table.to_csv(f"C:/Code/Python/Machine_Learning_AI/DealLI_Exceptions/Lookup_Table.csv")
del lookup_table

In [ ]:
subject = 'Lookup Table Created'
body = f'''
        Starting long category_new_values section.
    '''
# send_email_to_self(subject,body)

## Converting Categorical Data to Numerical

In [ ]:
category_new_values = {}
count = 0
for key,item in category_values.items():
    # print(f"Categories Read: {count}\nCategories Left: {len(category_values)-count}")
    count += 1
    for new_value in item:
        try:
            category_new_values[f"{key} : {new_value}"] = list(Series(list(train_subset[str(key)].unique())+list(train_subset[str(key)].unique())).unique()).index(new_value)
        except:
            category_new_values[f"{key} : {new_value}"] = len(list(Series(list(train_subset[str(key)].unique())+list(train_subset[str(key)].unique())).unique()))
            break
del category_values

In [ ]:
count = 0
for key,item in category_new_values.items():
    full_value = f"{key} : {item}"
    # print(f"Categories Read: {count}\nCategories Left: {len(category_new_values)-count}\n{full_value}\n")
    count += 1
    col = search(r"[A-Za-z\_\s]{1,}:",full_value).group()
    col = full_value.split(':')[0]
    # print(f"{col[:-1]}")
    col_value = search(":.+:",full_value).group()#search(r":[A-Za-z0-9\_\s\-\/\<\>\=\'\.\+\,\&]{1,}:",full_value).group()
    col_value = full_value.split(':')[1]
    # print(col_value[1:-1])
    new_value = search(r": [0-9]{1,}",full_value).group().replace(' ','').replace(':','')
    # print(new_value)
    # print()
    train_subset[col[:-1]] = train_subset[col[:-1]].replace({col_value[1:-1]: new_value})
del category_new_values

In [ ]:
subject = 'New categories completed'
body = f"""
        About to prepare for machine learning testing.
    """
# send_email_to_self(subject,body)

In [ ]:
for col in train_subset.columns:
    if train_subset[col].dtype == 'object':# and not(col=='checked'):
        try:
            train_subset[col] = train_subset[col].astype(float)
        except Exception as e:pass
            # train_subset = DataFrame(train_subset.drop(col,axis=1))
            # test = DataFrame(test.drop(col,axis=1))
            # print(f"{col}: {str(e)}")

## Datetime to Numerical

In [ ]:
datetime_columns = []
for col in train_subset.columns:
    if train_subset[col].dtype == 'datetime64[ns]':
        datetime_columns.append(col)
for col in datetime_columns:
    train_subset[col] = to_numeric(train_subset[col])
datetime_columns = []
del datetime_columns

## Machine Learning Preparation

In [ ]:
random_state = 2974306530
model_scores = DataFrame(None,columns=['Mean_Squared_Error','Root_Mean_Squared_Error','Mean_Absolute_Error','Median_Absolute_Error','Max_Error','R2'])
options.display.float_format = '{:.10f}'.format

In [ ]:
additional_excluded_columns_old = [
        'IsClaimFlg',
        'Checked',
        'ATG_Ref',
        'ATG_LI_Ref',
        'BatchNbr',
        'CATEGORY_ATG',
        'ClaimType',
        'AP_VndNbr',
        'APVndrName',
        'OOB_ATG',
        'ItemNbr',
        'UPCNbr',
        'UPCUnit',
        'ItemDescription',
        'ItemShipPack',
        'PoNbr',
        'PODate',
        'ReceivingDate',
        'TurnRatio_ATG',
        'DealNbr',
        'OrdStartDate_ATG',
        'OrdEndDate_ATG',
        'DateStartArrival_ATG',
        'DateEndArrival_ATG',
        'DLAmtOI',
        'DLAmtBB'
    ]

In [ ]:
unique_excluded_columns = []
for col in additional_excluded_columns:
    if col in unique_excluded_columns or not(col in list(train_subset.columns)):
        pass
    else:
        unique_excluded_columns.append(col)
additional_excluded_columns = unique_excluded_columns.copy()

In [ ]:
del unique_excluded_columns

In [ ]:
excluded_features = train_subset[additional_excluded_columns]

## Split Train/Test Data

In [ ]:
features = train_subset.drop(['PdOI_ATG']+additional_excluded_columns,axis=1)
target = train_subset['PdOI_ATG']

In [ ]:
features_train_subset,features_test,target_train_subset,target_test = train_test_split(features,target,random_state=random_state,test_size=0.5,shuffle=True)

In [ ]:
scaler = StandardScaler()
scaler.fit(features_train_subset)
features_train_subset_scaled = DataFrame(scaler.transform(features_train_subset),columns=features.columns)
features_test_scaled = DataFrame(scaler.transform(features_test),columns=features.columns)
target_test = target_test.reset_index(drop=True)

## ML Models

In [ ]:
dc = DummyRegressor(strategy='median')
dc,dc_train_predictions,dc_predictions,model_scores,dc_model_importances = run_and_analyze_model(dc,features_train_subset_scaled,target_train_subset,features_test_scaled,target_test,model_scores,'Dummy')
print(f"Dummy model predictions complete.")
subject = 'Dummy Model Predictions Complete'
body = f""
# send_email_to_self(subject,body)

In [ ]:
lr = LinearRegression()
lr,lr_train_predictions,lr_predictions,model_scores,lr_model_importances = run_and_analyze_model(lr,features_train_subset_scaled,target_train_subset,features_test_scaled,target_test,model_scores,'Linear')
print(f"Linear model predictions complete.")
subject = 'Linear Model Predictions Complete'
# send_email_to_self(subject,body)

In [ ]:
dt = DecisionTreeRegressor(criterion='absolute_error',random_state=random_state,max_depth=4,max_features=None)
dt,dt_train_predictions,dt_predictions,model_scores,dt_model_importances = run_and_analyze_model(dt,features_train_subset_scaled,target_train_subset,features_test_scaled,target_test,model_scores,'DecisionTree')
print(f"Decision Tree model predictions complete.")
subject = 'Decision Tree Model Predictions Complete'
# send_email_to_self(subject,body)

In [ ]:
rf = RandomForestRegressor(criterion='absolute_error',random_state=random_state,verbose=0,warm_start=True,max_depth=4,n_estimators=100)
rf,rf_train_predictions,rf_predictions,model_scores,rf_model_importances = run_and_analyze_model(rf,features_train_subset_scaled,target_train_subset,features_test_scaled,target_test,model_scores,'RandomForest')
print(f"Random Forest model predictions complete.")
subject = 'Random Forest Model Predictions Complete'
# send_email_to_self(subject,body)

In [ ]:
xgb = XGBRegressor(max_depth=6,random_state=random_state)
xgb,xgb_train_predictions,xgb_predictions,model_scores,xgb_model_importances = run_and_analyze_model(xgb,features_train_subset_scaled,target_train_subset,features_test_scaled,target_test,model_scores,'XGBoost')
print(f"XGB model predictions complete.")
subject = 'XGBoost Model Predictions Complete'
# send_email_to_self(subject,body)

In [ ]:
lgbm = LGBMRegressor(n_estimators=1000,learning_rate=0.01,random_state=random_state,verbose=-1)
lgbm,lgbm_train_predictions,lgbm_predictions,model_scores,lgbm_model_importances = run_and_analyze_model(lgbm,features_train_subset_scaled,target_train_subset,features_test_scaled,target_test,model_scores,'LGBM')
lgbm_model_importances['LGBM'] = (lgbm_model_importances['LGBM'])/(lgbm_model_importances['LGBM'].sum())
print(f"LGBM model predictions complete.")
subject = 'LGBM Model Predictions Complete'
# send_email_to_self(subject,body)

In [ ]:
gb = GradientBoostingRegressor(loss='absolute_error',warm_start=True,n_estimators=10000,learning_rate=0.0025,random_state=random_state,verbose=0,n_iter_no_change=3)
gb,gb_train_predictions,gb_predictions,model_scores,gb_model_importances = run_and_analyze_model(gb,features_train_subset_scaled,target_train_subset,features_test_scaled,target_test,model_scores,'GradientBoost')
print(f"Gradient Boost model predictions complete.")
subject = 'Gradient Boost Model Predictions Complete'
# send_email_to_self(subject,body)

In [ ]:
knn = KNeighborsRegressor(n_neighbors=3)
knn,knn_train_predictions,knn_predictions,model_scores,knn_model_importances = run_and_analyze_model(knn,features_train_subset_scaled,target_train_subset,features_test_scaled,target_test,model_scores,'KNeighbors')
print(f"KNeighbors model predictions complete.")
subject = 'KNearest Neighbors Predictions Complete'
# send_email_to_self(subject,body)

In [ ]:
cb = CatBoostRegressor(verbose=0,iterations=10000,learning_rate=0.0075,random_state=random_state,early_stopping_rounds=3)
cb,cb_train_predictions,cb_predictions,model_scores,cb_model_importances = run_and_analyze_model(cb,features_train_subset_scaled,target_train_subset,features_test_scaled,target_test,model_scores,'CatBoost')
cb_model_importances['CatBoost'] = (cb_model_importances['CatBoost'])/(cb_model_importances['CatBoost'].sum())
print(f"Cat Boost model predictions complete.")
subject = 'Cat Boost Model Predictions Complete'
# send_email_to_self(subject,body)

## Analyze Models

In [ ]:
model_scores.to_csv(f"C:/Code/Python/Machine_Learning_AI/DealLI_Exceptions/Model_Analysis/Model_Scores/Model_Scores_{upc}.csv")

In [ ]:
feature_importances = concat([dc_model_importances,lr_model_importances,dt_model_importances,rf_model_importances,
                            xgb_model_importances,gb_model_importances,lgbm_model_importances,
                            cb_model_importances,knn_model_importances],axis=1)
feature_importances['SUM'] = feature_importances['Dummy']+feature_importances['Linear']+feature_importances['DecisionTree']+feature_importances['RandomForest']+feature_importances['XGBoost']+feature_importances['GradientBoost']+feature_importances['LGBM']+feature_importances['CatBoost']+feature_importances['KNeighbors']

In [ ]:
feature_importances.to_csv(f'C:/Code/Python/Machine_Learning_AI/DealLI_Exceptions/Model_Analysis/Feature_Importances/Feature_Importances_{upc}.csv')

## Reconstructing Original Data With Predictions

In [ ]:
train_subset_original['PdOI_ATG'] = train_subset_original['PdOI_ATG'].astype(float)

In [ ]:
features_train_unscaled,features_test_unscaled = train_test_split(train_subset_original,random_state=random_state,test_size=0.5,shuffle=True)

In [ ]:
df_with_predictions_test = concat([DataFrame(features_test_unscaled).reset_index(drop=True),lr_predictions,dt_predictions,rf_predictions,xgb_predictions,gb_predictions,lgbm_predictions,cb_predictions,knn_predictions],axis=1)
df_with_predictions_train = concat([DataFrame(features_train_unscaled).reset_index(drop=True),
                                    lr_train_predictions,dt_train_predictions,
                                    rf_train_predictions,xgb_train_predictions,gb_train_predictions,
                                    lgbm_train_predictions,cb_train_predictions,knn_train_predictions],axis=1)

In [ ]:
df_with_predictions = concat([df_with_predictions_test,df_with_predictions_train],ignore_index=True,axis=0)

In [ ]:
options.display.float_format = '{:.4f}'.format

## Showing Discrepancies

In [ ]:
df_with_predictions.sort_values(by='UPCNbr').to_csv(f'Data_With_ML_Predictions.csv',index=False)
subject = 'Data with Model Predictions Added to CSV'
# send_email_to_self(subject,body)

In [ ]:
data = df_with_predictions[
                        (
                            (abs(df_with_predictions['PdOI_ATG']-df_with_predictions['Linear'])>=(df_with_predictions['PdOI_ATG']/5))&
                            (abs(df_with_predictions['PdOI_ATG']-df_with_predictions['DecisionTree'])>=(df_with_predictions['PdOI_ATG']/5))&
                            (abs(df_with_predictions['PdOI_ATG']-df_with_predictions['RandomForest'])>=(df_with_predictions['PdOI_ATG']/5))&
                            (abs(df_with_predictions['PdOI_ATG']-df_with_predictions['XGBoost'])>=(df_with_predictions['PdOI_ATG']/5))&
                            (abs(df_with_predictions['PdOI_ATG']-df_with_predictions['GradientBoost'])>=(df_with_predictions['PdOI_ATG']/5))&
                            (abs(df_with_predictions['PdOI_ATG']-df_with_predictions['LGBM'])>=(df_with_predictions['PdOI_ATG']/5))&
                            (abs(df_with_predictions['PdOI_ATG']-df_with_predictions['CatBoost'])>=(df_with_predictions['PdOI_ATG']/5))&
                            (abs(df_with_predictions['PdOI_ATG']-df_with_predictions['KNeighbors'])>=(df_with_predictions['PdOI_ATG']/5))&
                            (df_with_predictions['PdOI_ATG']<0.01)
                        )
                    ].reset_index(drop=True)
data.to_csv("C:/Code/Python/Machine_Learning_AI/DealLI_Exceptions/Discrepancies/All_Model_Predictions_Significantly_Different_From_Original_PdOI_ATG_Where_Pd_OI_Zero.csv",index=False)

In [ ]:
data_2 = df_with_predictions[
                        (
                            (df_with_predictions['Linear']>(df_with_predictions['PdOI_ATG']))&
                            (df_with_predictions['DecisionTree']>(df_with_predictions['PdOI_ATG']))&
                            (df_with_predictions['RandomForest']>(df_with_predictions['PdOI_ATG']))&
                            (df_with_predictions['XGBoost']>(df_with_predictions['PdOI_ATG']))&
                            (df_with_predictions['GradientBoost']>(df_with_predictions['PdOI_ATG']))&
                            (df_with_predictions['LGBM']>(df_with_predictions['PdOI_ATG']))&
                            (df_with_predictions['CatBoost']>(df_with_predictions['PdOI_ATG']))&
                            (df_with_predictions['KNeighbors']>(df_with_predictions['PdOI_ATG']))
                        )
                    ].reset_index(drop=True)
data_2.to_csv("C:/Code/Python/Machine_Learning_AI/DealLI_Exceptions/Discrepancies/All_Model_Predictions_Bigger_Than_Original_PdOI_ATG.csv",index=False)

In [ ]:
data_3 = df_with_predictions[
                        (
                            (abs(df_with_predictions['PdOI_ATG']-df_with_predictions['Linear'])>=(df_with_predictions['PdOI_ATG']/5))&
                            (abs(df_with_predictions['PdOI_ATG']-df_with_predictions['DecisionTree'])>=(df_with_predictions['PdOI_ATG']/5))&
                            (abs(df_with_predictions['PdOI_ATG']-df_with_predictions['RandomForest'])>=(df_with_predictions['PdOI_ATG']/5))&
                            (abs(df_with_predictions['PdOI_ATG']-df_with_predictions['XGBoost'])>=(df_with_predictions['PdOI_ATG']/5))&
                            (abs(df_with_predictions['PdOI_ATG']-df_with_predictions['GradientBoost'])>=(df_with_predictions['PdOI_ATG']/5))&
                            (abs(df_with_predictions['PdOI_ATG']-df_with_predictions['LGBM'])>=(df_with_predictions['PdOI_ATG']/5))&
                            (abs(df_with_predictions['PdOI_ATG']-df_with_predictions['CatBoost'])>=(df_with_predictions['PdOI_ATG']/5))&
                            (abs(df_with_predictions['PdOI_ATG']-df_with_predictions['KNeighbors'])>=(df_with_predictions['PdOI_ATG']/5))&
                            (df_with_predictions['PdOI_ATG']>0.01)
                        )
                    ].reset_index(drop=True)
data_3.to_csv("C:/Code/Python/Machine_Learning_AI/DealLI_Exceptions/Discrepancies/All_Model_Predictions_Significantly_Different_From_Original_PdOI_ATG_Where_Pd_OI_NOT_Zero.csv",index=False)

In [ ]:
print(f"{len(df_with_predictions):,} rows being exported to SQL.\n")
export_to_sql(df_with_predictions)